In [14]:
from dateutil.relativedelta import relativedelta
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras import Model, regularizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, BatchNormalization, Dot
from joblib import Parallel, delayed
from keras.models import Model
from datetime import datetime
from typing import Dict, List
import scipy.stats as stats
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
import itertools
import random
import pickle
import time
import gc


In [15]:
def dropRowsAndColsForCA(df: pd.DataFrame, lhs_col: str) -> pd.DataFrame:
   # drop 2018-2019 and 2022 data
   # - '18-'19 does not have enough assets
   # - 2022 is oos for now
   df = df[~df.date.dt.year.isin([2018, 2019, 2022])].reset_index(drop=True)

   # Set characteristics of interest
   selected_rhs = ['char_addr_active_tm1h', 'char_bidask_t', 'char_circulation_dormant_tm365', 
      'char_delta_flow_dist_tm1h', 'char_dex_prct_circ_supply_t', 'char_exchange_inflow_tm1h', 
      'char_exchange_outflow_tm1h', 'char_exchange_prct_circ_supply_t', 'char_mvrv_t', 
      'char_num_pairs_t', 'char_prct_supply_in_profit_t', 'char_r_ath_t', 'char_r_atl_t', 'char_r_industry_tm6h', 
      'char_r_max_tm12h', 'char_r_tm1h', 'char_sent_volume_consumed_tm1',
      'char_size_realized_t', 'char_social_volume_tm7', 'char_trades_t', 
      'char_turnover_tm7', 'char_tx_volume_t', 'char_vol_tm1']

   # Cut to characteristics columns of interest
   df = df[['date', 'asset', lhs_col]+selected_rhs]

   # Note: keep obs to RHS ratio roughly 4e4:1

   # Note: for any macro, take cartesian product with characteristics to make it characteritisc level
   # -or do the reg thing to reduce it down to same dim as number of assets

   return df

In [16]:
def formPortfolioReturnCovariates(df: pd.DataFrame, lhs_col: str) -> pd.DataFrame:
    # Obtain the datetimes of the dataframe
    df = df.sort_values(by = 'date')
    datetimes = np.unique(df.date.values)

    # Form new covariate names
    characteristics = list(df.columns.values)
    characteristics.remove('date')
    characteristics.remove('asset')
    characteristics.remove(lhs_col)
    new_covars = ['x_' + char[5:] for char in characteristics]

    # Loop over all datetimes
    for current_dt in datetimes: 
        # Obtain the datetime's LHS "tomorrow" returns and the covariates
        r_tp1 = df[df.date == current_dt].r_ex_tp1.values
        z_t  = df[df.date == current_dt][characteristics].values
        
        # Calculate the characteristic managed portfolio returns
        design = np.linalg.inv(np.matmul(np.transpose(z_t), z_t))
        x_tp1    = np.matmul(np.matmul(design, np.transpose(z_t)), r_tp1)
        
        # Set the new columns to this week's vector's value
        df.loc[df.date == current_dt, new_covars] = x_tp1

    return df


In [17]:
def buildAutoencoder(b_covars, x_covars, 
    number_hidden_layer, l1_penalty, weight_initializer, bias_initializer, number_factor, learning_rate):
    # Build the betas model from the time t covariates
    model_b = tf.keras.models.Sequential()
    model_b.add(tf.keras.Input(shape=(len(b_covars),)))
    for j in range(number_hidden_layer):
        model_b.add(Dense(16*1/(2**(j)), activation='relu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model_b.add(BatchNormalization())
    model_b.add(Dense(number_factor, activation='linear',
                    kernel_initializer=weight_initializer,
                    bias_initializer=bias_initializer))

    # Form the x model from time t plus 1 returns
    model_x = tf.keras.models.Sequential()
    model_x.add(tf.keras.Input(shape=(len(x_covars),)))
    model_x.add(Dense(number_factor, activation='linear',
                    kernel_initializer=weight_initializer,
                    bias_initializer=bias_initializer))

    # Form the dot product output for the combination of the two neurals
    mergedOut = Dot(axes=(1,1))([model_b.output, model_x.output])

    # Form the entire model
    model = Model([model_b.input, model_x.input], mergedOut)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss='mean_squared_error',
                    metrics=['mse'])

    return model

def fitAutoencoder(train_df: pd.DataFrame, 
    hps_dict: dict, val_df: pd.DataFrame=None, lhs_col: str='r_ex_tp1', rhs_cols: list=[], num_cpus: int=-1) -> list:
    # Obtain beta and factor side covariates
    b_covars = [covar for covar in rhs_cols if covar[:4] == 'char']
    x_covars = [covar for covar in rhs_cols if covar[:2] == 'x_']
    assert set(rhs_cols) == (set(b_covars).union(set(x_covars)))

    # Extract the hyperparameters
    number_hidden_layer = hps_dict['num_hidden_layers']
    number_factor       = hps_dict['number_factors']
    learning_rate       = hps_dict['learning_rates']
    l1_penalty          = hps_dict['l1_penalties']
    batch_size          = hps_dict['batch_sizes']
    number_ensemble     = hps_dict['num_ensemble']
    epoch               = hps_dict['epochs']
    early_stopping      = hps_dict['early_stopping']

    # Loop over the ensembles to build models for each
    models = []
    assert(number_ensemble <= 10), 'whatcha think you got infinite come pew ters'
    for i in range(0, number_ensemble):
        # Obtain the training input and output data and, if passed, validation data
        train_b = train_df[b_covars]
        train_x = train_df[x_covars]  
        train_y = train_df[[lhs_col]]
        if val_df is not None:
            val_b = val_df[b_covars]
            val_x = val_df[x_covars]  
            val_y = val_df[[lhs_col]]

        # According to which model in the ensemble it is, initialize parameters.
        random.seed(i*42)
        initializer_list = [initializers.HeNormal(seed=i), 
                            initializers.GlorotUniform(seed=i), 
                            initializers.RandomUniform(seed=i)]
        initializer_pair = random.sample(initializer_list, 2)
        weight_initializer = initializer_pair[0]
        bias_initializer   = initializer_pair[1]

        # Build the model
        model = buildAutoencoder(b_covars, x_covars, number_hidden_layer, 
            l1_penalty, weight_initializer, bias_initializer, number_factor, learning_rate)

        # Prepare early stopping object 
        es = EarlyStopping(monitor='val_mse', mode='min', verbose=2, patience = 5) 

        # Fit the model
        with tf.device('/CPU:0'):
            if early_stopping == True:
                model.fit(x=[train_b, train_x], y=train_y, 
                            batch_size=batch_size,
                            validation_data=([val_b, val_x], val_y), 
                            epochs=epoch, verbose=2,
                            workers=20, callbacks=[es])  # TODO CHANGE WORKERS TO 1 WHEN I WRAP UP
            else:
                model.fit(x=[train_b, train_x], y=train_y, 
                            batch_size=batch_size,
                            epochs=epoch, verbose=2,
                            workers=20)  # TODO CHANGE WORKERS TO 1 WHEN I WRAP UP
                
        models.append(model)

    # build the time window for the training data to predict on
    first_datetime = np.min(train_df.date.values)
    month = np.datetime64(first_datetime, 'M').astype(int) % 12 + 1
    year = np.datetime64(first_datetime, 'Y').astype(int) + 1970
    oos_start_date = np.min(train_df[(train_df.date.dt.year==year) 
                        & (train_df.date.dt.month==month+1)].date.values)
    oos_end_date   = np.max(train_df.date.values)

    # fit on the training data for all models to report the r2_pred
    train_yhats = genAutoencoderYhats(train_df, oos_start_date, oos_end_date, number_factor, num_cpus)
    train_ys    = train_df[train_df.date>=oos_start_date][lhs_col].values
    train_r2_pred = 1-(np.mean(np.square(train_ys-train_yhats)))/(np.mean(np.square(train_ys)))

    return models, train_r2_pred

def genAutoencoderYhats(df, oos_start_date, oos_end_date, number_factor, num_cpus) -> np.array:
    # Obtain beta and factor side covariates
    b_covars = [covar for covar in rhs_cols if covar[:4] == 'char']
    x_covars = [covar for covar in rhs_cols if covar[:2] == 'x_']
    assert set(rhs_cols) == (set(b_covars).union(set(x_covars)))

    # Obtain the oos data
    oos_df = df[(df.date >= oos_start_date) & (df.date <= oos_end_date)].copy()
    oos_x  = oos_df[x_covars]
    oos_b  = oos_df[b_covars]

    # Form each model's results
    b_hats = np.zeros((oos_df.shape[0], number_factor, len(models)))
    lambda_hats = np.zeros((oos_df.shape[0], number_factor, len(models)))
    all_dates = np.unique(df[(df.date <= oos_end_date)].date)
    oos_dates = np.unique(oos_df.date.values)
    for i in range(len(models)):
        print(i)
        # Update the model to use
        model = models[i]

        # Form the beta hats
        layer_name = model.layers[-3]._name  
        assert(model.layers[-3].output_shape[1] == number_factor)
        b_hat_layer = Model(inputs=model.input[0],
                            outputs=model.get_layer(layer_name).output)
        b_hat = b_hat_layer.predict(oos_b, verbose=0)
        b_hats[:,:,i] = b_hat

        # Form the sample average of the estimated factors up to each oos date
        # build this model's mapping from input to f hat
        model = models[i]
        layer_name = model.layers[-2]._name 
        assert(model.layers[-2].output_shape[1] == number_factor)
        f_hat_layer = Model(inputs=model.input[1],
                            outputs=model.get_layer(layer_name).output)

        # estimate this model's f hats for all dates before the oos dates
        x = df[df.date < oos_start_date][x_covars]
        f_hat = f_hat_layer.predict(x, verbose=0)
        assert(all(np.isclose(f_hat[0,:], f_hat[1,:])))
        f_hats = np.sum(f_hat, axis=0)

        # obtain the f hats for the entire oos period
        x = df[df.date >= oos_start_date][x_covars]
        f_hat = f_hat_layer.predict(x, verbose=0)
        assert(all(np.isclose(f_hat[0,:], f_hat[1,:])))

        # determine the lambda hat for each oos date
        lambda_hat_index_start = 0
        for t in range(len(oos_dates)):
            # update this oos date 
            oos_date = oos_dates[t]

            # update the fhats with the appropriate f_hat values
            # -update start index in here so this is skipped on first run but occurs
            #  on every other before we update the end index two lines below
            if t != 0:
                f_hats += np.sum(f_hat[lambda_hat_index_start:lambda_hat_index_end,:], axis=0)
                lambda_hat_index_start = lambda_hat_index_end

            # determine how many obs are in this oos date
            num_rows_in_oos_dt = df[df.date==oos_date].shape[0]
            
            # update the end index given the number of oos obs for this date
            lambda_hat_index_end = lambda_hat_index_start + num_rows_in_oos_dt

            # divide by total number of f_hats added together to figure out TS average for this oos_date
            #     save as this time period's and this model's lambda hat
            lambda_hats[lambda_hat_index_start:lambda_hat_index_end, :, i] = (
                np.tile(f_hats / df[df.date<oos_date].shape[0], (num_rows_in_oos_dt, 1)))
            
    # Form model predictions of beta hats times lambda hats where
    #     we take dot product between two factor length vectors for all time periods and models
    #     and then average each model's forecast to return a vector of length of oos dataframe
    yhats = np.mean(np.sum(b_hats * lambda_hats, axis=1), axis=1)

    return yhats


In [18]:
def plotYvsYhat(y: np.ndarray, yhats: np.ndarray, num_quantiles: int = 10) -> None:
    """ Plot the average values of y (returns) against bins of yhat (predicted returns).

    Args:
        y (np.ndarray): A vector of target variable.
        yhats (np.ndarray): A vector of fitted values.
        num_quantiles (int): The number of quantiles to bin yhats into. Default is 10.
    """
    # Create a DataFrame with y and yhats
    temp_df = pd.DataFrame({'y': y, 'yhat': yhats})

    # Bin yhats into quantiles
    temp_df['yhat'] = pd.qcut(temp_df['yhat'], q=num_quantiles, labels=False)

    # Group y by yhat and plot the means
    temp_df.groupby('yhat')['y'].mean().plot()

    # Set plot title
    plt.title('Mean Return vs. Quantiles of Predicted Returns')

    # Show the plot
    plt.show()


In [19]:
def GenTestYhats(df, opt_hps, test_year=2021): 
    test_weeks = np.unique(df[df.index.year == test_year].index.values)
    
    test_df = pd.DataFrame()

    for test_week in test_weeks:
        # TODO DO SOME LOGIC TO FIGURE OUT THE WINDOWS TO FIT A MODEL FOR SO I LOOP OVER THOSE INITIAL DATES
        # -fit teh model once
        # -then predict on all subsequent weeks
        # -maybe one month at time?
        # -maybe one week at a time?
        # -let's try once a month to see how it does
        print(test_week, '\n')
        temp_df  = df[df.index <= test_week].copy()
        temp_df  = subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week=test_week)
        train_df = temp_df[temp_df.index < test_week].copy()
        oos_df   = temp_df[temp_df.index == test_week].copy()
        
        models   = fitAutoencoder(train_df, opt_hps, val_df=None, early_stopping=False)
        yhats    = genYhats(oos_df, models, test_week, opt_hps['number_factor'])
       
        oos_df = oos_df[['asset', 'r_tplus7']]
        oos_df['yhat'] = yhats
        test_df = pd.concat((test_df, oos_df))
        rw_mse = np.mean(np.square(test_df.r_tplus7.values))
        model_mse = np.mean(np.square(test_df.r_tplus7.values - test_df.yhat.values))
        print('\n test random walk mse: ' + str(rw_mse))
        print('\n test model mse: ' + str(model_mse))
        print('winning?: ' + str(model_mse < rw_mse))
        print('\n')
    
    return test_df


In [20]:
def predictTestPeriod(df: pd.DataFrame,
    lhs_col: str,
    test_start_date: str,
    test_end_date: str,
    opt_model_dict: dict,
    num_cpus: int) -> np.array:
    ''' predict for test period using backtested model.

    Args:
        df (pd.DataFrame): panel of training and test data with date index, LHS variable named 
                           `lhs_col`, and remaining cols are RHS features.
        lhs_col (str): name of the lhs target column.
        test_start_date (str): the first date for the test period.
        test_end_date (str): the last date for the test period.
        opt_model_dict (dict): hyperparameters for optimal backtested model.
        num_cpus (int): number of cpus to use when parallelizing.
    
    Returns:
        yhats (np.array): vector of yhat positions for entire test period.
    '''
    # form list of rhs col
    rhs_cols = list(df.columns.values)
    rhs_cols.remove(lhs_col)

    # form lhs for oos df
    oos_df = df.copy()
    ret_threshold = opt_model_dict['ret_threshold']
    oos_df['y'] = 1
    oos_df.loc[oos_df[lhs_col] > ret_threshold, 'y'] = 2
    oos_df.loc[oos_df[lhs_col] < -ret_threshold, 'y'] = 0

    # form test dates to loop over
    test_dates = np.unique(oos_df[test_start_date:test_end_date].index.values)

    def loopOverTestDates(test_date):
        # form train and val data
        temp_df   = oos_df[oos_df.index <= test_date].copy()
        train_df  = temp_df[temp_df.index < test_date].copy()
        train_y_btc_eth_diff = train_df[lhs_col].values
        train_df  = train_df.drop(lhs_col, axis=1)
        test_df   = temp_df[temp_df.index == test_date].copy()
        test_df   = test_df.drop(lhs_col, axis=1)

        # fit and predict
        model, train_acc, train_yhats = fitRF(train_df, 'y', rhs_cols, 
                                            hps=opt_model_dict, 
                                            ys_real=train_y_btc_eth_diff)
        yhats = genRFYhats(test_df, model, 'y', rhs_cols)

        return yhats, model, train_acc, train_yhats

    # predict for entire test period
    test_results = Parallel(n_jobs=int(num_cpus/4))(delayed(loopOverTestDates)(test_date) for test_date in test_dates)

    # extract test yhats to return
    yhats_list = []
    for t in range(len(test_results)):
        yhats_list.append(test_results[t][0])
    yhats = np.array(yhats_list)-1

    return yhats

In [21]:
def reportTestPeriodResults(yhats: np.array, y_real: np.array):
    ''' report various portfolio statistics for the test period.
    
    Args:
        yhats (np.array): vector of actual portfolio positions.
        y_real (np.array): vector of real return difference for target instrument.
    '''
    # calculate returns post transaction costs
    tc = calcTransactionCosts(yhats)
    returns = yhats*y_real - tc
    
    # calc portfolio statistics
    geom_mean = calcGeomAvg(returns)
    sharpe    = calcSharpe(returns, periods_in_year=365*12)
    max_dd    = calcMaxDrawdown(returns)
    max_1week = calcMaxOneWeekLoss(returns, periods_in_week=7*12)

    # report
    print(f"Geometric average 2 hour return: {geom_mean:.6f}")
    print(f"Annualized Sharpe: {sharpe:.3f}")
    print(f"Maximum drawdown: {max_dd:.4f}")
    print(f"Maximum loss over any one week period: {max_1week:.4f}")

    # plot classification
    plotYvsYhat(y=y_real, yhats=yhats)

In [9]:
if __name__ == "__main__":
    # set settings
    #tf.config.set_visible_devices([], 'GPU') # TODO CONFIRM THIS RUNS IT ON GPU

    # set args
    IN_FP           = '../data/clean/panel_train.pkl'
    ASSET_IN_FP     = '../data/clean/asset_universe_dict.pickle'
    CV_OUT_FP       = '../output/cv_results'
    LHS_COL         = 'r_ex_tp1'
    VAL_START_DATE  = '2021-01-01'
    VAL_END_DATE    = '2021-06-30'
    TEST_START_DATE = '2021-07-01'
    NUM_CPUS        = 22 # TODO change to 22 when doing on desktop
    PERIODS_IN_YEAR = 365*24
    HP_GRID         = {'num_hidden_layers': [3, 2, 1],
        'number_factors': [6, 5, 4, 3, 2, 1],
        'learning_rates': [1e-4, 1e-3, 1e-2, 1e-1],
        'batch_sizes': [1024],
        'l1_penalties': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
        'num_ensemble': [10],
        'early_stopping': [True],
        'epochs': [100]}
    
    # read in data
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    df = pd.read_pickle(IN_FP)

    # drop rows and columns such that data will work for conditional autoencoder (CA)
    df = dropRowsAndColsForCA(df, LHS_COL)

    # form the char-sorted portfolios for factor side of CA input
    df = formPortfolioReturnCovariates(df, LHS_COL) # NOTE: ~7 min runtime


In [10]:
# TODO CV FUNCTION
# def runCV(df: pd.DataFrame, asset_universe_dict: Dict[str, List])

# WHAT TO RETURN:

# ARGS TO PASS IN:
df
asset_universe_dict
val_start_date = VAL_START_DATE
val_end_date   = VAL_END_DATE
test_start_date = TEST_START_DATE
lhs_col = LHS_COL
hp_grid = HP_GRID
#model_params = PARAMS
#fitModel = FUNC
#predictModel = FUNC
num_cpus = NUM_CPUS
periods_in_year = PERIODS_IN_YEAR
cv_out_fp = CV_OUT_FP
arch_name = 'autoencoder'

In [11]:
# Subset to relevant data
df = df[df.date < test_start_date].copy()

# Initialize cv result objects
results_list = []

# Determine RHS columns
rhs_cols = list(df.columns.values)
rhs_cols.remove('date')
rhs_cols.remove('asset')
rhs_cols.remove(lhs_col)

# Determine validation datetimes to loop over and datetimes to refit at
val_dts_dict = {}
val_datetimes = np.unique(df[df.date>=val_start_date].date.values)
val_sun_midnights = np.unique(df[(df.date>=val_start_date) 
    & (df.date.dt.hour==0) & (df.date.dt.day_of_week==6)].date.values)

# Check if first val date is sunday midnight, if not then add the dates
first_val_date = np.min(df[(df.date==val_start_date)].date.values)
day_of_week_of_first_val_datetime = (first_val_date.astype('datetime64[D]').view('int64') - 4) % 7
if day_of_week_of_first_val_datetime != 6:
    val_dts_dict[first_val_date] = np.unique(df[(df.date>=first_val_date) & (df.date<val_sun_midnights[0])].date.values)

# Complete the dictionary with all the sundays as keys as the dates until the next sunday as the values
for val_sun_midnight in val_sun_midnights:
    next_sun_midnight = val_sun_midnight + np.timedelta64(7, 'D')
    val_dts_dict[val_sun_midnight] = np.unique(df[(df.date>=val_sun_midnight) 
                                        & (df.date<next_sun_midnight)
                                        & (df.date<test_start_date)].date.values)



In [13]:
# Loop over hp combinations
keys = hp_grid.keys()
values = hp_grid.values()
hp_combos = list(itertools.product(*values))
for hps in hp_combos:
    # Start the timer
    tic = time.perf_counter()
    
    # Create hp dictionary and other objects for this iteration
    hps_dict = dict(zip(keys, hps))
    hps_results_dict = hps_dict.copy()
    val_y_yhats_df = df[(df.date >= val_start_date) & (df.date < test_start_date)][['date', 'asset', lhs_col]].copy()

    # Report on progress
    print(hps_dict)

    # Define function to loop over
    train_r2_pred_list = []
    for val_datetime_start in list(val_dts_dict.keys()):
        # form end of this window
        val_datetime_end = np.max(val_dts_dict[val_datetime_start])

        # form appropriate asset universe
        first_day_of_month_for_current_val_dt = np.datetime_as_string(val_datetime_start, unit='M')+'-01'
        asset_universe = asset_universe_dict[first_day_of_month_for_current_val_dt]

        # form train and val dataframes
        temp_df = df[(df.date<=val_datetime_end) & (df.asset.isin(asset_universe))].copy()
        train_df = temp_df[temp_df.date<val_datetime_start].copy()
        val_df = temp_df[(temp_df.date>=val_datetime_start) & (temp_df.date<=val_datetime_end)].copy()
        del temp_df
        gc.collect()

        # fit and predict
        models, train_r2_pred = fitAutoencoder(train_df, hps_dict, val_df, lhs_col, rhs_cols, num_cpus)
        yhats = genAutoencoderYhats(df, 
            val_datetime_start, val_datetime_end, hps_dict['number_factors'], num_cpus)

        # save the results
        train_r2_pred_list.append(train_r2_pred)
        temp_yhats_df = val_df[['date', 'asset']].copy()
        temp_yhats_df['yhats'] = yhats
        val_y_yhats_df = val_y_yhats_df.merge(temp_yhats_df, on=['date', 'asset'], how='left', validate='one_to_one')

    # Stop the timer
    toc = time.perf_counter()

    break


{'num_hidden_layers': 3, 'number_factors': 6, 'learning_rates': 0.0001, 'batch_sizes': 1024, 'l1_penalties': 0.1, 'num_ensemble': 10, 'early_stopping': True, 'epochs': 100}


2023-05-02 19:23:45.738561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5354 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:65:00.0, compute capability: 8.6


Epoch 1/100


2023-05-02 19:23:47.716909: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:347 : INVALID_ARGUMENT: Trying to access resource Resource-21-at-0x274cf450 (defined @ /home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/keras/engine/base_layer_utils.py:134) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-05-02 19:23:47.716989: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:347 : INVALID_ARGUMENT: Trying to access resource Resource-22-at-0x2764fa70 (defined @ /home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/keras/engine/base_layer_utils.py:134) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-05-02 1

InvalidArgumentError: Graph execution error:

Detected at node 'StatefulPartitionedCall_12' defined at (most recent call last):
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/adam/miniconda3/envs/tf/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/ho

In [8]:


# -form metadata: val_start_date, val_end_date, arch_name, runtime
# -form training stats: some avg of the r2_pred
# -form val period stats: r2_pred, geom avg ret, sharpe_anual, std_annual, ts_avg_annual, avg_yhat, min, q1, q2, q3, max yhat,  max dd, avg turnover

# -save the cv results to csv

# return the list of dictionaries of results
# return results_list


In [ ]:

hps_yhats_dict['yhats'] = np.append(hps_yhats_dict['yhats'], yhats)
hps_yhats_dict['ys']    = np.append(hps_yhats_dict['ys'], ys)

val_ys_todate = hps_yhats_dict['ys']
rw_val_mse    = np.mean(np.square(val_ys_todate))
model_val_mse = np.mean(np.square(val_ys_todate - hps_yhats_dict['yhats']))
#             print('\n val random walk mse: ' + str(rw_val_mse))
#             print('\n val model mse: ' + str(model_val_mse))
#             print('\n val model mse winning?: ' + str(model_val_mse < rw_val_mse))
#             print('\n\n')
# TODO WRITE FUNC HERE TO REPORT SIMPLE STUFF LIKE DIST OF PREDICTED RETURNS AND PREDICTIVE R^2
# Save run time and space out result print out
toc = time.perf_counter()
#         print('\n\n\n')

# Update lists of results
# TODO WRITE FUNC TO REPORT MORE STUFF HERE FROM OTHER NOTEBOOKS
# -including all the results that i will have for test period
hps_yhats_dict_list.append(hps_yhats_dict)
cv_results_dict = hps_yhats_dict.copy()
del cv_results_dict['yhats']
del cv_results_dict['ys']
cv_results_dict['runtime_mins'] = round((toc - tic)/60, 0)
cv_results_dict['mse'] = model_val_mse
hps_mse_df_list.append(pd.DataFrame(cv_results_dict, index=[0]))

# Save CV results
cv_df = pd.concat(hps_mse_df_list, ignore_index=True)
timestr = time.strftime("%Y%m%d_%H%M%S")
fp = '../4-output/cv-results-autoencoder-' + timestr + '.csv'
cv_df.to_csv(fp, index=False)




# extract validation periods results
yhats_list = []
models_list = []
train_r2_pred_list = []
train_yhats_list = []
for t in range(len(val_results)):
    yhats_list.append(val_results[t][0])
    models_list.append(val_results[t][1])
    train_r2_pred_list.append(val_results[t][2])
    train_yhats_list.append(val_results[t][3])
yhats = np.array(yhats_list)
positions = np.sign(yhats)
ys    = cv_df[val_start_date:][lhs_col].values
results_dict['yhats'] = yhats

# save results to master result list    
results_list.append(results_dict)

# save results to csv to monitor during cv
toc = time.perf_counter()
csv_dict = results_dict['hps'].copy()
del results_dict
csv_dict['arch_name'] = arch_name
csv_dict['val_start_date'] = val_start_date
csv_dict['val_end_date'] = np.datetime_as_string(np.max(cv_df.index.values))[:10]
csv_dict['runtime_mins'] = round((toc - tic)/60, 0) 
csv_dict['mean_r2_pred_train'] = np.mean(np.array(train_r2_pred_list))
csv_dict['r2_pred_val'] = 1-np.mean(np.square(ys-yhats))/np.mean(np.square(ys))
tc = calcTransactionCosts(positions)
returns = positions*ys - tc
csv_dict['geom_mean_1h'] = calcGeomAvg(returns)
csv_dict['sharpe_ann'] = calcSharpe(returns, periods_in_year=periods_in_year)
csv_dict['sd_ann'] = calcSD(returns, annualized=True, periods_in_year=periods_in_year)
csv_dict['ts_mean_ann'] = calcTSAvgReturn(returns, annualized=True, periods_in_year=periods_in_year)
for lev in [2, 3, 4]:
    lev_pos = positions*lev
    lev_tc = calcTransactionCosts(lev_pos)
    lev_returns = lev_pos*ys - lev_tc 
    csv_dict['geom_mean_x'+str(lev)] = calcGeomAvg(lev_returns)
    csv_dict['sharpe_x'+str(lev)] = calcSharpe(lev_returns, periods_in_year=periods_in_year)
    csv_dict['sd_ann_x'+str(lev)] = calcSD(lev_returns, annualized=True, periods_in_year=periods_in_year)
    csv_dict['ts_mean_ann_x'+str(lev)] = calcTSAvgReturn(lev_returns, annualized=True, periods_in_year=periods_in_year)
csv_dict_list.append(csv_dict)
results_df = pd.DataFrame(csv_dict_list)

timestr = time.strftime("%Y%m%d_%H%M%S")
fp = out_fp+'_'+arch_name+'_'+timestr+'.csv'
results_df.to_csv(fp, index=False)

# output results to track
print(csv_dict['runtime_mins'])
print(f"val r2_pred: {csv_dict['r2_pred_val']:.4f}")
print(f"val sharpe: {csv_dict['sharpe_ann']:.4f}")

print('\n\n')
gc.collect()



In [194]:
# TODO write to file the beta and factor side values each time i predict?


In [ ]:
# TODO THIS NOTEBOOK ENDS WITH SAVING A DF OF DATE, ASSET, PREDICTED RETURN FOR A VAL OR TEST PERIOD

In [ ]:
# TODO GO SCOPE OLD NOTEBOOK FOR LATEST NUMBERS

In [ ]:
# TODO ALSO NEED TO TRY A MODEL WHERE WE SET MISSING ASSETS LHS 
# AND RHS TO -2 AND THEN WE HAVE BALANCED PANEL SO WE FIT ON THE 
# MATRIX OF RHS ACROSS ASSETS AS OPPOSED OT INDIVIDUAL DATE-ASSET 
# AND THEN THE NETWORK LEARNS ASSET-SPECIFIC PARAMETERS